<a href="https://colab.research.google.com/github/mohithxoxo/NLP/blob/master/NLP_Models(classification_problem).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [37]:
train = pd.read_csv('/content/train.txt',header=None,sep=';')
validation = pd.read_csv('/content/val.txt',header=None,sep=';')
test = pd.read_csv('/content/test.txt',header=None,sep=';')
train = train.append(test)

In [38]:
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()
train[1] = le.fit_transform(train[1])
test[1] = le.fit_transform(test[1])
validation[1] = le.fit_transform(validation[1])

In [39]:
train

,0,1
0,i didnt feel humiliated,4
1,i can go from feeling so hopeless to so damned...,4
2,im grabbing a minute to post i feel greedy wrong,0
3,i am ever feeling nostalgic about the fireplac...,3
4,i am feeling grouchy,0
...,...,...
1995,i just keep feeling like someone is being unki...,0
1996,im feeling a little cranky negative after this...,0
1997,i feel that i am useful to my people and that ...,2
1998,im feeling more comfortable with derby i feel ...,2


In [65]:
train[1].value_counts()

2    6057
4    5247
0    2434
1    2161
3    1463
5     638
Name: 1, dtype: int64

In [40]:
#We will check the maximum number of words that can be present in a comment , this will help us in padding later
train[0].apply(lambda x:len(str(x).split())).max()

66

In [41]:
xtrain, xtest, ytrain, ytest = train_test_split(train[0].values, train[1].values, 
                                                  stratify=train[1].values, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

# **A simpleRNN without any pretrained embeddings and one dense layer**

In [42]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 1500

token.fit_on_texts(list(xtrain) + list(xtest))
xtrain_seq = token.texts_to_sequences(xtrain)
xtest_seq = token.texts_to_sequences(xtest)

#zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xtest_pad = sequence.pad_sequences(xtest_seq, maxlen=max_len)

word_index = token.word_index

In [43]:
%%time
with strategy.scope():
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     input_length=max_len))
    model.add(SimpleRNN(100))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1500, 300)         4855500   
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 100)               40100     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 4,895,701
Trainable params: 4,895,701
Non-trainable params: 0
_________________________________________________________________
CPU times: user 218 ms, sys: 39.3 ms, total: 257 ms
Wall time: 309 ms


In [48]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=4*strategy.num_replicas_in_sync) 
#Multiplying by Strategy to run on TPU's

Epoch 1/5
  2/450 [..............................] - ETA: 30s - loss: -3546.5894 - accuracy: 0.2031WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0038s vs `on_train_batch_end` time: 0.0459s). Check your callbacks.


450/450 [==============================] - 22s 49ms/step - loss: -3046.9041 - accuracy: 0.2138
Epoch 2/5
450/450 [==============================] - 22s 49ms/step - loss: -3120.6655 - accuracy: 0.2141
Epoch 3/5
450/450 [==============================] - 22s 49ms/step - loss: -3179.1724 - accuracy: 0.2103
Epoch 4/5
450/450 [==============================] - 22s 48ms/step - loss: -3250.1250 - accuracy: 0.2125
Epoch 5/5
450/450 [==============================] - 22s 48ms/step - loss: -3313.0757 - accuracy: 0.2144


# **NLP with LSTM's**

We have already tokenized and paded our text for input to LSTM's

The latest approach to getting word Embeddings is using pretained GLoVe or using Fasttext.You can download the GloVe vectors from here http://www-nlp.stanford.edu/data/glove.840B.300d.zip or you can search for GloVe in datasets on Kaggle and add the file

In [ ]:
!wget http://www-nlp.stanford.edu/data/glove.840B.300d.zip 
#2gb file

In [58]:
!unzip /content/glove.840B.300d.zip

Archive:  /content/glove.840B.300d.zip
  inflating: glove.840B.300d.txt     


In [59]:
# load the GloVe vectors in a dictionary:

embeddings_index = {}
f = open('/content/glove.840B.300d.txt','r',encoding='utf-8')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(val) for val in values[1:]])
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

2196017it [03:41, 9898.45it/s] 

Found 2196016 word vectors.


In [60]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 16184/16184 [00:00<00:00, 238086.54it/s]


In [61]:
%%time
with strategy.scope():
    
    # A simple LSTM with glove embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))

    model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1500, 300)         4855500   
_________________________________________________________________
lstm (LSTM)                  (None, 100)               160400    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 5,016,001
Trainable params: 160,501
Non-trainable params: 4,855,500
_________________________________________________________________
CPU times: user 676 ms, sys: 274 ms, total: 950 ms
Wall time: 2.51 s


In [63]:
model.fit(xtrain_pad, ytrain, epochs=10, batch_size=4*strategy.num_replicas_in_sync)

Epoch 1/10
  2/450 [..............................] - ETA: 4:30 - loss: 0.2957 - accuracy: 0.1094WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0018s vs `on_train_batch_end` time: 0.0873s). Check your callbacks.


450/450 [==============================] - 41s 91ms/step - loss: -48.2800 - accuracy: 0.1202
Epoch 2/10
450/450 [==============================] - 40s 88ms/step - loss: -112.6651 - accuracy: 0.1201
Epoch 3/10
450/450 [==============================] - 40s 88ms/step - loss: -175.0192 - accuracy: 0.1201
Epoch 4/10
450/450 [==============================] - 40s 88ms/step - loss: -237.0228 - accuracy: 0.1201
Epoch 5/10
450/450 [==============================] - 40s 88ms/step - loss: -298.8703 - accuracy: 0.1201
Epoch 6/10
450/450 [==============================] - 40s 88ms/step - loss: -360.6262 - accuracy: 0.1201
Epoch 7/10
450/450 [==============================] - 40s 88ms/step - loss: -422.3293 - accuracy: 0.1201
Epoch 8/10
450/450 [==============================] - 40s 89ms/step - loss: -483.9839 - accuracy: 0.1201
Epoch 9/10
450/450 [==============================] - 40s 89ms/step - loss: -545.6653 - accuracy: 0.1201
Epoch 10/10
450/450 [==============================] - 40s 89ms/ste

# **Gated Recurrent Unit**

 simpler and faster than LSTM's

In [66]:
%%time
with strategy.scope():
    # GRU with glove embeddings and two dense layers
     model = Sequential()
     model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
     model.add(SpatialDropout1D(0.3))
     model.add(GRU(300))
     model.add(Dense(1, activation='sigmoid'))

     model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])   
    
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1500, 300)         4855500   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 1500, 300)         0         
_________________________________________________________________
gru (GRU)                    (None, 300)               541800    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 301       
Total params: 5,397,601
Trainable params: 542,101
Non-trainable params: 4,855,500
_________________________________________________________________
CPU times: user 449 ms, sys: 208 ms, total: 657 ms
Wall time: 3.44 s


In [70]:
model.fit(xtrain_pad, ytrain, epochs=10, batch_size=4*strategy.num_replicas_in_sync)

Epoch 1/10
  2/450 [..............................] - ETA: 4:30 - loss: -152.7733 - accuracy: 0.1875WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0040s vs `on_train_batch_end` time: 0.0947s). Check your callbacks.


450/450 [==============================] - 46s 102ms/step - loss: -238.7662 - accuracy: 0.1201
Epoch 2/10
450/450 [==============================] - 45s 101ms/step - loss: -424.9332 - accuracy: 0.1201
Epoch 3/10
450/450 [==============================] - 45s 100ms/step - loss: -609.4783 - accuracy: 0.1201
Epoch 4/10
450/450 [==============================] - 45s 100ms/step - loss: -793.6362 - accuracy: 0.1201
Epoch 5/10
450/450 [==============================] - 45s 100ms/step - loss: -977.8021 - accuracy: 0.1201
Epoch 6/10
450/450 [==============================] - 45s 100ms/step - loss: -1161.7516 - accuracy: 0.1201
Epoch 7/10
450/450 [==============================] - 45s 100ms/step - loss: -1345.8352 - accuracy: 0.1201
Epoch 8/10
450/450 [==============================] - 45s 100ms/step - loss: -1529.7795 - accuracy: 0.1201
Epoch 9/10
450/450 [==============================] - 45s 100ms/step - loss: -1713.5156 - accuracy: 0.1201
Epoch 10/10
450/450 [==============================] 

# **Bi-Directional RNN's**

In [71]:
%%time
with strategy.scope():
    # A simple bidirectional LSTM with glove embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
    model.add(Bidirectional(LSTM(300, dropout=0.3, recurrent_dropout=0.3)))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    
    
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 1500, 300)         4855500   
_________________________________________________________________
bidirectional (Bidirectional (None, 600)               1442400   
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 601       
Total params: 6,298,501
Trainable params: 1,443,001
Non-trainable params: 4,855,500
_________________________________________________________________
CPU times: user 769 ms, sys: 194 ms, total: 962 ms
Wall time: 4.54 s


In [73]:
model.fit(xtrain_pad, ytrain, epochs=10, batch_size=2*strategy.num_replicas_in_sync)

Epoch 1/10
  2/450 [..............................] - ETA: 9:56 - loss: -1.0641 - accuracy: 0.1406 WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0039s vs `on_train_batch_end` time: 0.4416s). Check your callbacks.


450/450 [==============================] - 201s 447ms/step - loss: -224.0658 - accuracy: 0.1201
Epoch 2/10
450/450 [==============================] - 200s 444ms/step - loss: -594.4350 - accuracy: 0.1201
Epoch 3/10
450/450 [==============================] - 200s 444ms/step - loss: -961.5794 - accuracy: 0.1201
Epoch 4/10
 44/450 [=>............................] - ETA: 3:00 - loss: -1148.6790 - accuracy: 0.1286

KeyboardInterrupt: ignored

# **BERT**

In [ ]:
!pip install ktrain
from ktrain import text 
import ktrain
import pandas as pd

In [83]:
train = train.rename(columns={0:'text',1:'label'})
trn, val, preproc = text.texts_from_df(train,'text', label_columns=['label'],preprocess_mode='bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [84]:
model = text.text_classifier('bert', trn, preproc=preproc)
learner = ktrain.get_learner(model,train_data=trn, val_data=val, batch_size=4)

Is Multi-Label? False
maxlen is 400
done.


In [ ]:
learner.fit_onecycle(2e-5, 2)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/2
 434/4050 [==>...........................] - ETA: 3:08:05 - loss: 1.5178 - accuracy: 0.4332

begin training using onecycle policy with max lr of 2e-05...
Epoch 1/2
4050/4050 [==============================] - 13233s 3s/step - loss: 0.5792 - accuracy: 0.7902 - val_loss: 0.1999 - val_accuracy: 0.9161
Epoch 2/2
Epoch 2/2
2504/4050 [=================>............] - ETA: 1:21:17 - loss: 0.1459 - accuracy: 0.9396